## Data Extraction 

In [ ]:
# import global modules
import os
import sys
import pandas as pd
import re
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_util_data = pth_project / 'core' / 'utils' / 'data'
pth_queries = pth_project / 'core' / 'utils' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

In [ ]:
bq_client =  connect_bq_services(d_project_config['gcp-project-name'])

In [ ]:
%load_ext autoreload
%autoreload 2

#### 1. Extract & format conversation

In [ ]:
# Query conversations
sql = (pth_queries / 'common' / 'conv_extract.sql').read_text().format(
    max_date='2022-04-11', min_date='2022-04-11'
)
df_conv = extract_bq_data(bq_client, sql)

# load province code
d_pr_codes = extract_pr_codes(pth_util_data / 'cnty_codes.csv')

In [ ]:
df_conv_formated = format_conv_df(
    df_conv, d_pr_codes, d_params['num_col']
)[d_params['conv_col_formatted']]

#### 2 Extract, format and filter sentences

In [ ]:
sql = (pth_queries / 'common'  / 'sentence_extract.sql').read_text()
sql = sql.format(
    allowed_lang=d_params['allowed_lang'], min_exchange=d_params['min_exchange'], 
    max_date='2022-04-11', min_date='2022-04-11'
)
df_sntces = extract_bq_data(bq_client, sql)

In [ ]:
df_sntces

#### 3. Filtering and saving elligible conversation

In [ ]:
df_filtered_conv, df_filtered_sntces = filter_convs(
    df_conv_formated, df_sntces, d_params['conv_id_col']
)
df_filtered_conv.to_csv(pth_data / 'extract' / 'conversations.csv', index=False)
df_filtered_sntces.to_csv(pth_data / 'extract' / 'sentences.csv', index=False)

print(f'{df_filtered_conv.shape[0]} elligible conversation have been saved')
print(f'{df_filtered_sntces.shape[0]} elligible sentences have been saved')

 ### EDA

In [ ]:
df_sntces.head()

In [ ]:
df_conv.head()

In [ ]:
df_conv_formated.columns

In [ ]:
df_conv_formated.pr_name.unique()

In [ ]:
conv_id_col=d_params['conv_id_col']

In [ ]:
for i, (pr, df_sub) in enumerate(df_conv_formated.groupby('pr_name')):
    if i>0:break
    print(pr,df_sub.shape)

In [ ]:
d_sizing = {"Province": pr, 'Total conv': len(df_sub)}

In [ ]:
df_sub = df_sub.loc[df_sub[conv_id_col].isin(df_sntces[conv_id_col].unique())]

In [ ]:
df_sub.shape

#### 4. Sizing analysis

In [ ]:
def compute_sizing(df_conv, df_sntnces, conv_id_col):
    l_sizing = []
    # For each province
    for pr, df_sub in df_conv.groupby('pr_name'):
        d_sizing = {"Province": pr, 'Total conv': len(df_sub)}
        
        # Filter out non english / noisy sentences
        df_sub = df_sub.loc[df_sub[conv_id_col].isin(df_sntnces[conv_id_col].unique())]
        d_sizing['Conv in english'] = len(df_sub)
        
        # Filter out low iss
        df_sub = df_sub.loc[(~df_sub.iss_max.isnull()) & (df_sub.iss_max > 0.75)]
        d_sizing['Conv with high iss'] = len(df_sub)
        
        # Filter out missign num
        df_sub = df_sub.loc[~df_sub.num.isnull()]
        d_sizing['Conv with known identification'] = len(df_sub)
        
        # Add to list
        l_sizing.append(d_sizing)
        
    # For all data
    d_sizing = {"Province": "ALL", 'Total conv': len(df_conv)}
                
    # Filter out non english / noisy sentences
    df_conv_sub = df_conv.loc[df_conv[conv_id_col].isin(df_sntnces[conv_id_col].unique())]
    d_sizing['Conv in english'] = len(df_conv_sub)

    # Filter out low iss
    df_conv_sub = df_conv_sub.loc[(~df_conv_sub.iss_max.isnull()) & (df_conv_sub.iss_max > 0.75)]
    d_sizing['Conv with high iss'] = len(df_conv_sub)

    # Filter out missign num
    df_conv_sub = df_conv_sub.loc[~df_conv_sub.num.isnull()]
    d_sizing['Conv with known identification'] = len(df_conv_sub)
    l_sizing.append(d_sizing)

    return pd.DataFrame(l_sizing)
            

In [ ]:
df_sizing = compute_sizing(df_conv_formated, df_sntces, d_params['conv_id_col'])

In [ ]:
df_sizing 

#### 5. Other KPIs

In [ ]:
df_filtered_sntces.groupby('call_convrstn_id').agg({"sntnce": 'count'}).sort_values(by='sntnce').describe()